In [ ]:
server_name = 'tcp:sjobaccount1.database.windows.net,1433'
database = 'dbjobaccount'
username = 'cloudsa'
passwd = 'Yukon900!'
job_cred_name=input('new job credential name:')
job_credential = input('new job scoped credential user:')
job_passwd = input('new job scoped credential password:')

target_cred_name = input('new target credential name:')
target_user = input('new target scoped credential user:')
target_passwd = input('new target scoped credential password:')

In [1]:
# Connect to server
import pyodbc 
import pandas as pd
import numpy as np

server_name = 'tcp:sjobaccount1.database.windows.net,1433'
database = 'dbjobaccount'
username = 'cloudsa'
passwd = 'Yukon900!'

jobsDbConnection = pyodbc.connect('Driver={ODBC Driver 13 for SQL Server};'
                        'UID='+username+';'
                        'PWD='+passwd+';'
                      'Server='+server_name+';'
                      'Database='+database+';'
                      'Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;'
                      )

print("Connected to "+server_name+" successfully")

Connected to tcp:sjobaccount1.database.windows.net,1433 successfully


In [14]:
#note : this should be done in the target server
#refresh credentials need to be in master


masterDbConnection = pyodbc.connect('Driver={ODBC Driver 13 for SQL Server};'
                        'UID='+username+';'
                        'PWD='+passwd+';'
                      'Server='+server_name+';'
                      'Database=master;'
                      'Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;'
                      )

masterCursor = masterDbConnection.cursor()

refresh_name = input('enter new refresh credentials name:')
refresh_user = input('enter refreshcredential user:')
refresh_passwd = input('enter new refreshcredential password:')

jobcredentialuser= input('enter new job credential login:')
jobcredentialpassword= input('enter new job credential password:')



queries =[
    "CREATE LOGIN "+refresh_name+" WITH PASSWORD='"+refresh_passwd+"'",
    "CREATE USER "+refresh_user+" FROM LOGIN "+refresh_name,
    "CREATE LOGIN "+jobcredentialuser+" WITH PASSWORD='"+jobcredentialpassword+"'"
]

for query in queries:
    masterCursor.execute(query)

enter new refresh credentials name:refreshcred
enter refreshcredential user:refreshcred
enter new refreshcredential password:myPassw0rd!
enter new job credential login:jobcred
enter new job credential password:myPassw0rd!


In [20]:
#Then, on EACH of the target databases, you need to connect and issue the following: 

queries =[    
    "CREATE USER "+jobcredentialuser+" FROM LOGIN "+jobcredentialuser,    
    "GRANT CREATE TABLE TO "+jobcredentialuser+";"
]

for query in queries:
    masterCursor.execute(query)

ProgrammingError: ('42000', '[42000] [Microsoft][ODBC Driver 13 for SQL Server][SQL Server]Grantor does not have GRANT permission. (4613) (SQLExecDirectW)')

In [27]:
#Connect to the job database specified when creating the job agent
database = 'dbjobaccount'

cursor = jobsDbConnection.cursor()

master_key_encryption_passwd=input('new master key encryption password:')

job_cred_name=input('new job credential name:')
job_credential = input('new job scoped credential user:')
job_passwd = input('new job scoped credential password:')

refresh_cred_name = input('new refresh credential name:')
refresh_user = input('new refresh scoped credential user:')
refresh_passwd = input('new refresh scoped credential password:')

queries =[
    "IF NOT EXISTS (SELECT * FROM sys.symmetric_keys WHERE name LIKE '%DatabaseMasterKey%')"+
    "BEGIN "+        
        "CREATE MASTER KEY ENCRYPTION BY PASSWORD = '"+master_key_encryption_passwd+"' "+
    "END",
    # Create a db master key if one does not already exist, using your own password.      
    #"CREATE MASTER KEY ENCRYPTION BY PASSWORD='"+master_key_encryption_passwd+"';",
    # Create a database job credential.  
    "CREATE DATABASE SCOPED CREDENTIAL "+job_cred_name+" WITH IDENTITY = '"+job_credential+"', SECRET = '"+job_passwd+"';",
    # Create a database refresh credential for the master database of server1.
    "CREATE DATABASE SCOPED CREDENTIAL "+refresh_cred_name+" WITH IDENTITY = '"+refresh_user+"',SECRET = '"+refresh_passwd+"';"
]

for query in queries:
    cursor.execute(query)

new master key encryption password:j0bpassw0rd!
new job credential name:jobcred
new job scoped credential user:jobcred
new job scoped credential password:myPassw0rd!
new refresh credential name:refreshcred
new refresh scoped credential user:refreshcred
new refresh scoped credential password:myPassw0rd!
